In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
df=pd.read_csv("Iris.csv")
df=df.drop(columns=["Id"], errors='ignore')
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
le=LabelEncoder()
df['Species']=le.fit_transform(df['Species'])


In [ ]:
X=df.drop(columns=["Species"]).values
y=df['Species'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [ ]:
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Convert to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Custom Dataset
class IrisDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y
  def __len__(self):
    return len(self.X)
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]


In [ ]:
# DataLoader
train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 16)

In [ ]:
for x, y in train_loader:
  print(x.shape, y.shape)
  break

torch.Size([16, 4]) torch.Size([16])


In [ ]:
# Model
class IrisClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(4,16),
        nn.ReLU(),
        nn.Linear(16,16),
        nn.ReLU(),
        nn.Linear(16,3)
    )
  def forward(self, x):
    return self.net(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IrisClassifier().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Training Loop

epochs=50
for epoch in range(epochs):
  model.train()
  total_loss = 0

  for X_batch, y_batch in train_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    optimizer.zero_grad()
    outputs = model(X_batch)
    loss = criterion(outputs, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  if (epoch + 1) % 10 == 0:
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 10/50, Loss: 0.6800
Epoch 20/50, Loss: 0.4686
Epoch 30/50, Loss: 0.4283
Epoch 40/50, Loss: 0.3872
Epoch 50/50, Loss: 0.5726


In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
  for X_batch, y_batch in test_loader:
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    outputs = model(X_batch)
    _, predicted = torch.max(outputs, 1)

    total += y_batch.size(0)
    correct += (predicted == y_batch).sum().item()
accuracy = 100 * correct / total
print(f"Test Accuracy:{accuracy:.2f}%")

Test Accuracy:100.00%
